In [4]:
import pandas as pd

transactions = pd.read_csv('../all_transactions.csv')
transfers = pd.read_csv('../all_transfers.csv')

trans_data = transactions[['client_code', 'date', 'avg_monthly_balance_KZT', 'amount']].copy()
trans_data['source'] = 'transaction'
trans_data['direction'] = 'out'

balance_map = transactions.groupby('client_code')['avg_monthly_balance_KZT'].first().to_dict()

transf_data = transfers[['client_code', 'date', 'amount', 'direction']].copy()
transf_data['source'] = 'transfer'
transf_data['avg_monthly_balance_KZT'] = transf_data['client_code'].map(balance_map)

freezable_deposit_data = pd.concat([trans_data, transf_data], ignore_index=True)

freezable_deposit_data

,client_code,date,avg_monthly_balance_KZT,amount,source,direction
0,10,2025-06-01 08:10:05,1277325.0,3338.23,transaction,out
1,10,2025-06-01 12:10:01,1277325.0,27765.82,transaction,out
2,10,2025-06-01 19:00:55,1277325.0,21966.75,transaction,out
3,10,2025-06-03 09:15:48,1277325.0,5217.89,transaction,out
4,10,2025-06-03 12:10:20,1277325.0,12925.04,transaction,out
...,...,...,...,...,...,...
35395,9,2025-08-30 13:31:36,3084180.0,33934.54,transfer,out
35396,9,2025-08-30 20:31:07,3084180.0,23968.31,transfer,out
35397,9,2025-08-30 21:14:53,3084180.0,13317.67,transfer,out
35398,9,2025-08-31 11:11:25,3084180.0,21562.19,transfer,out


In [ ]:
import pandas as pd
import json
from datetime import datetime

# Load config
with open('product_config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

# Load data files
clients_df = pd.read_csv('../clients.csv')
transactions_df = pd.read_csv('../all_transactions.csv')
transfers_df = pd.read_csv('../all_transfers.csv')

# Convert dates
transactions_df['date'] = pd.to_datetime(transactions_df['date'])
transfers_df['date'] = pd.to_datetime(transfers_df['date'])

# Create product-specific dataframes
product_dfs = {}

for product_name, product_config in config.items():
    print(f"\nProcessing {product_name}...")
    
    # Start with empty list to collect all data
    all_product_data = []
    
    # Get client data
    base_clients = clients_df.copy()
    
    # Apply status filter to clients if specified
    if 'status' in product_config.get('filters', {}):
        base_clients = base_clients[base_clients['status'].isin(product_config['filters']['status'])]
    
    # Process transfers if in sources
    if 'all_transfers' in product_config['sources']:
        transfers_temp = transfers_df.copy()
        
        # Merge with client data
        transfers_temp = transfers_temp.merge(base_clients, on='client_code', how='inner')
        
        # Apply filters
        if 'type' in product_config.get('filters', {}):
            transfers_temp = transfers_temp[transfers_temp['type'].isin(product_config['filters']['type'])]
        
        if 'direction' in product_config.get('filters', {}):
            transfers_temp = transfers_temp[transfers_temp['direction'].isin(product_config['filters']['direction'])]
        
        if 'currency' in product_config.get('filters', {}):
            transfers_temp = transfers_temp[transfers_temp['currency'].isin(product_config['filters']['currency'])]
        
        # Add source column
        transfers_temp['data_source'] = 'transfer'
        transfers_temp['category'] = None  # Add empty category column for consistency
        
        all_product_data.append(transfers_temp)
    
    # Process transactions if in sources
    if 'all_transactions' in product_config['sources']:
        trans_temp = transactions_df.copy()
        
        # Merge with client data
        trans_temp = trans_temp.merge(base_clients, on='client_code', how='inner')
        
        # Apply filters
        if 'category' in product_config.get('filters', {}):
            trans_temp = trans_temp[trans_temp['category'].isin(product_config['filters']['category'])]
        
        if 'currency' in product_config.get('filters', {}):
            trans_temp = trans_temp[trans_temp['currency'].isin(product_config['filters']['currency'])]
        
        # Add source column and missing columns for consistency
        trans_temp['data_source'] = 'transaction'
        trans_temp['type'] = None  # Add empty type column for consistency
        trans_temp['direction'] = None  # Add empty direction column for consistency
        
        all_product_data.append(trans_temp)
    
    # Combine all data
    if all_product_data:
        combined_df = pd.concat(all_product_data, ignore_index=True)
        
        # Sort by client and date
        combined_df = combined_df.sort_values(['client_code', 'date'])
        
        # Reorder columns for consistency
        column_order = ['client_code', 'name', 'status', 'age', 'city', 'avg_monthly_balance_KZT', 
                       'date', 'data_source', 'type', 'category', 'direction', 'amount', 'currency']
        
        # Only include columns that exist
        final_columns = [col for col in column_order if col in combined_df.columns]
        combined_df = combined_df[final_columns]
        
        product_dfs[product_name] = combined_df
        print(f"Created {product_name} dataframe with {len(combined_df)} rows")
        print(f"Unique clients: {combined_df['client_code'].nunique()}")
    else:
        print(f"No data for {product_name}")

# Save each product dataframe
for product_name, df in product_dfs.items():
    output_file = f'../separate_dfs/{product_name}_data.csv'
    df.to_csv(output_file, index=False)
    print(f"\nSaved {output_file} with shape: {df.shape}")


Processing deposit_multicurrency...
Created deposit_multicurrency dataframe with 24447 rows
Unique clients: 60

Processing deposit_savings...
Created deposit_savings dataframe with 19110 rows
Unique clients: 57

Processing deposit_accumulative...
Created deposit_accumulative dataframe with 6380 rows
Unique clients: 44

Processing investments...
Created investments dataframe with 6943 rows
Unique clients: 44

Processing gold_bars...
Created gold_bars dataframe with 3854 rows
Unique clients: 16

Saved deposit_multicurrency_data.csv with shape: (24447, 13)

Saved deposit_savings_data.csv with shape: (19110, 13)

Saved deposit_accumulative_data.csv with shape: (6380, 13)

Saved investments_data.csv with shape: (6943, 13)

Saved gold_bars_data.csv with shape: (3854, 13)
